<a href="https://colab.research.google.com/github/ifran-rahman/Federated-ECG/blob/master/FL_Simulation_TNR_Lab_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🔗 Mount Google Drive (if needed in Colab)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


📦 Install Required Packages

In [ ]:
!pip install h5py
!pip install typing-extensions
!pip install wheel
!pip install -U flwr["simulation"]

📚 Imports

In [ ]:
import time
import tracemalloc
import sys
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, SubsetRandomSampler, random_split
from torch import nn, optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import flwr as fl
from collections import OrderedDict
from typing import Dict, List, Tuple, Optional, Union
from flwr.common import NDArrays, Scalar, Metrics

🧠 Memory and Device Setup

In [ ]:
tracemalloc.start()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

🧪 Dataset Preparation Function

In [ ]:

def prepare__dataset(abnormal: pd.DataFrame, normal: pd.DataFrame, val_split_factor: float) -> tuple[torch.utils.data.Dataset, torch.utils.data.Dataset, dict]:
    """Prepares the training and validation datasets for ECG classification, shuffling the data before splitting.

    Args:
        abnormal (pd.DataFrame): DataFrame containing abnormal ECG data.
        normal (pd.DataFrame): DataFrame containing normal ECG data.
        val_split_factor (float): Fraction of the data to use for validation.

    Returns:
        tuple[torch.utils.data.Dataset, torch.utils.data.Dataset, dict]: A tuple containing the training dataset,
            the validation dataset, and a dictionary with the number of examples in each.
    """
    abnormal = abnormal.drop([187], axis=1)
    normal = normal.drop([187], axis=1)

    y_abnormal = np.ones((abnormal.shape[0]))
    y_abnormal = pd.DataFrame(y_abnormal)

    y_normal = np.zeros((normal.shape[0]))
    y_normal = pd.DataFrame(y_normal)

    x = pd.concat([abnormal, normal], sort=True)
    y = pd.concat([y_abnormal, y_normal], sort=True)

    x = x.to_numpy()
    y = y[0].to_numpy()

    # Create a TensorDataset before shuffling
    full_dataset = torch.utils.data.TensorDataset(torch.from_numpy(x).float(),
                                                  torch.from_numpy(y).long())

    # Calculate the lengths for training and validation sets
    full_len = len(full_dataset)
    val_len = int(full_len * val_split_factor)
    train_len = full_len - val_len

    # Shuffle the dataset using a random permutation of indices
    indices = torch.randperm(full_len).tolist()
    train_indices = indices[:train_len]
    val_indices = indices[train_len:]

    # Create SubsetRandomSamplers to get shuffled subsets
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

    # Create DataLoaders using the samplers
    train_dataset = torch.utils.data.DataLoader(full_dataset, batch_size=train_len, sampler=train_sampler)
    val_dataset = torch.utils.data.DataLoader(full_dataset, batch_size=val_len, sampler=val_sampler)

    num_examples = {'trainset': train_len,
                    'testset': val_len}

    # Extract the datasets from the DataLoaders (since DataLoader returns iterators)
    train_dataset = list(train_dataset)[0]
    val_dataset = list(val_dataset)[0]

    return train_dataset, val_dataset, num_examples

🧩 CNN Model for ECG Classification

In [ ]:
class ecg_net(nn.Module):

    def __init__(self, num_of_class):
        super(ecg_net, self).__init__()

        self.model = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),

            nn.Conv1d(16, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),

            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),

        )

        self.linear = nn.Sequential(
            nn.Linear(2944,500),
            nn.LeakyReLU(inplace=True),
            nn.Linear(500, num_of_class),

        )


    def forward(self,x):
        x = x.unsqueeze(1)
        x = self.model(x)
        # print(x.shape)
        x = x.view(x.size(0), -1)
        #x [b, 2944]
        # print(x.shape)
        x = self.linear(x)

        return x

📈 Evaluation Function

In [ ]:
def evalute(model, loader):
    model.eval()
    correct = 0
    total = len(loader)
    val_bar = tqdm(loader, file=sys.stdout)
    for x, y in val_bar:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
        correct += torch.eq(pred, y).sum().float().item()
    return correct / total

🏋️‍♂️ Local Training Function

In [ ]:
def train_client(model, train_loader, valid_loader, epochs=1):
    optimizer = optim.Adam(model.parameters(), lr=3e-3)
    criteon = nn.CrossEntropyLoss()
    best_acc, best_epoch, global_step = 0, 0, 0
    for epoch in range(epochs):
        train_bar = tqdm(train_loader, file=sys.stdout)
        for step, (x, y) in enumerate(train_bar):
            x, y = x.to(device), y.to(device)
            model.train()
            logits = model(x)
            loss = criteon(logits, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_bar.desc = f"train epoch[{epoch+1}/{epochs}] loss:{loss:.3f}"
            global_step += 1
        val_acc = evalute(model, valid_loader)
        if val_acc > best_acc:
            best_epoch = epoch
            best_acc = val_acc
    print('best acc:', best_acc, 'best epoch:', best_epoch)

In [ ]:
"""## One Client, One Data Partition

To start designing a Federated Learning pipeline we need to meet one of the key properties in FL: each client has its own data partition. To accomplish this with the dataset, we are going to generate N random partitions, where N is the total number of clients in our FL system.
"""

from torch.utils.data import random_split


def get_dataset(abnormal, normal, val_split_factor): # done

    abnormal = abnormal.drop([187], axis=1)
    normal = normal.drop([187], axis=1)

    y_abnormal = np.ones((abnormal.shape[0]))
    y_abnormal = pd.DataFrame(y_abnormal)

    y_normal = np.zeros((normal.shape[0]))
    y_normal = pd.DataFrame(y_normal)

    x = pd.concat([abnormal, normal], sort=True)
    y = pd.concat([y_abnormal, y_normal] ,sort=True)

    x = x.to_numpy()
    y = y[0].to_numpy()

    train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(x).float(),
                                                torch.from_numpy(y).long())

    train_len = x.shape[0]
    val_len = int(train_len * val_split_factor)
    train_len -= val_len

    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

    num_examples =  {'trainset': train_len,
                    'testset': val_len}

    return train_dataset, val_dataset, num_examples

🧠 Flower Client

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, trainloader, vallodaer) -> None:
        super().__init__()
        self.trainloader = trainloader       # Local training DataLoader
        self.valloader = vallodaer           # Local validation DataLoader
        self.model = ecg_net(2).to(device)   # Initialize model and move to device (CPU/GPU)

    def set_parameters(self, parameters):
        # Convert received NumPy arrays into model state_dict format
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
        self.model.load_state_dict(state_dict, strict=True)  # Load weights into the model

    def get_parameters(self, config: Dict[str, Scalar]):
        # Convert model weights from tensors to NumPy arrays to send to the server
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def fit(self, parameters, config):
        # Receive updated global parameters and train locally
        self.set_parameters(parameters)
        train_client(self.model, self.trainloader, self.valloader, epochs=5)
        # Return updated local weights and the number of training examples
        return self.get_parameters({}), len(self.trainloader), {}

    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar]) -> Tuple[float, int, Dict[str, Scalar]]:
        # Load latest global parameters
        self.set_parameters(parameters)
        criterion = nn.CrossEntropyLoss()
        total_loss = 0.0
        correct = 0
        total = 0
        self.model.eval()
        # Run evaluation on local validation set
        with torch.no_grad():
            for inputs, labels in self.valloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        avg_loss = total_loss / len(self.valloader)  # Average loss per batch
        accuracy = correct / total                   # Total accuracy
        return avg_loss, total, {"accuracy": accuracy}


📊 Server Evaluation Function

In [ ]:
# Initialize the global model
model = ecg_net(2).to(device)

# Define the server-side evaluation function
def evaluate(server_round, parameters, config):
    # Convert received parameters from server (NumPy arrays) to PyTorch state_dict
    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)  # Load weights into the model

    # Loss function for evaluation
    criterion = nn.CrossEntropyLoss()

    # Tracking variables for total loss and accuracy
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    model.eval()  # Set model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation for efficiency
        for valloader in valloaders:  # Iterate over all client validation loaders
            for inputs, labels in valloader:
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Compute loss
                loss = criterion(outputs, labels)
                total_loss += loss.item() * inputs.size(0)  # Scale by batch size

                # Compute predictions and count correct ones
                _, predicted = torch.max(outputs, 1)
                total_correct += (predicted == labels).sum().item()
                total_samples += labels.size(0)

    # Calculate average loss and accuracy over all validation data
    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples

    # Return the evaluation metrics as required by Flower
    return avg_loss, {"accuracy": accuracy}

💾 Save Model Strategy

In [ ]:
# Custom FedAvg strategy that saves the aggregated global model after each round
class SaveModelStrategy(fl.server.strategy.FedAvg):

    def aggregate_fit(self, server_round, results, failures):
        # Perform standard FedAvg aggregation from Flower
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)

        # If aggregation was successful
        if aggregated_parameters is not None:
            print(f"Saving round {server_round} aggregated weights...")

            # Convert Flower Parameters object to list of NumPy arrays
            aggregated_ndarrays = fl.common.parameters_to_ndarrays(aggregated_parameters)

            # Save the aggregated weights as a .npz file (easily reloadable)
            np.savez(f"round-{server_round}-weights.npz", *aggregated_ndarrays)

            # Convert NumPy arrays to PyTorch state_dict format
            params_dict = zip(model.state_dict().keys(), aggregated_ndarrays)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})

            # Load aggregated weights into the global model
            model.load_state_dict(state_dict, strict=True)

            # Save the full PyTorch model checkpoint
            torch.save(model.state_dict(), f"model_round_{server_round}.pth")

        # Return the aggregated parameters and metrics back to Flower
        return aggregated_parameters, aggregated_metrics

⚙️ FL Config and Client Generator

In [ ]:
# Function to customize training configuration for each round
def fit_config(server_round):
    # Return training configuration as a dictionary
    # Use 1 local epoch for the first round, then increase to 2
    return {
        "batch_size": 1,
        "local_epochs": 1 if server_round < 2 else 2
    }

# Function to compute a weighted average of accuracy across all clients
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply each client's accuracy by its number of examples
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    # Extract the number of examples per client
    examples = [num_examples for num_examples, _ in metrics]
    # Return the global (weighted) accuracy
    return {
        "accuracy": sum(accuracies) / sum(examples)
    }



# Factory function to generate Flower clients based on client ID
def generate_client_fn(trainloaders, valloaders):
    # Inner function that Flower will call to instantiate clients
    def client_fn(cid: str):
        # Use the client ID to select corresponding train/val loaders
        return FlowerClient(
            trainloader=trainloaders[int(cid)],
            vallodaer=valloaders[int(cid)]
        )
    return client_fn

🚀 Federated Training

In [ ]:
# Set the number of participating clients
NUM_CLIENTS = 2

# Prepare client datasets (train/val) and a test set (if needed)
# NOTE: `prepare_dataset` must be defined to return loaders for each client
trainloaders, valloaders, testloader = prepare_dataset(num_partitions=NUM_CLIENTS, batch_size=20, val_ratio=0.1)

# Generate a function that returns the appropriate FlowerClient instance based on client ID
client_fn_callback = generate_client_fn(trainloaders, valloaders)

# Define the federated averaging strategy and specify custom behavior
strategy = SaveModelStrategy(
    fraction_fit=1.0,                  # Use all clients for training in each round
    min_fit_clients=2,                # Minimum number of clients required to train
    min_available_clients=2,          # Minimum clients that must be available to start the round
    evaluate_fn=evaluate,             # Custom server-side evaluation function
    on_fit_config_fn=fit_config       # Send training config to each client per round
)

# Start timing the training
start_training = time.time()

# Launch the Flower federated simulation
fl.simulation.start_simulation(
    client_fn=client_fn_callback,                     # Client selection function
    num_clients=NUM_CLIENTS,                          # Total number of simulated clients
    config=fl.server.ServerConfig(num_rounds=3),      # Number of global training rounds
    strategy=strategy,                                # Federated strategy with model saving
    client_resources={"num_cpus": 1, "num_gpus": 1},  # Resource allocation per client
    ray_init_args={"log_to_driver": False, "num_cpus": 1, "num_gpus": 1}  # Ray backend config
)

# End timing
end_training = time.time()

# Print total training duration
print('Total Training Time:', end_training - start_training)

# Report memory usage
current, peak = tracemalloc.get_traced_memory()
print(f'Current memory [MB]: {current/(1024*1024):.2f}, Peak memory [MB]: {peak/(1024*1024):.2f}')
tracemalloc.stop()

TypeError: prepare_dataset() got an unexpected keyword argument 'num_partitions'

In [ ]:
# -*- coding: utf-8 -*-
"""
FL Simulation TNR Lab with CNN
Cleaned and commented version of original Colab-generated script
"""

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install required packages
!pip install h5py
!pip install typing-extensions
!pip install wheel
!pip install -U flwr["simulation"]

# Imports
import time
import tracemalloc
import sys
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, SubsetRandomSampler, random_split
from torch import nn, optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import flwr as fl
from collections import OrderedDict
from typing import Dict, List, Tuple, Optional, Union
from flwr.common import NDArrays, Scalar, Metrics

# Start memory tracking
tracemalloc.start()

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Prepare ECG dataset for training and validation
def prepare_dataset(num_partitions: int, batch_size: int, val_ratio: float = 0.1):
    """This function partitions the training set into N disjoint
    subsets, each will become the local dataset of a client. This
    function also subsequently partitions each training set partition
    into train and validation. The test set is left intact and will
    be used by the central server to asses the performance of the
    global model."""

    # get the datatset
    trainset, testset, _ = get_dataset(abnormal=abnormal, normal=normal, val_split_factor=val_split_factor)

    print(len(trainset))
    print(len(testset))
    # split trainset into `num_partitions` trainsets
    num_images = len(trainset) // num_partitions

    partition_len = [num_images] * num_partitions

    partition_len[len(partition_len)-1] = len(trainset)-sum(partition_len[:-1])
    print(partition_len)
    trainsets = random_split(
        trainset, partition_len, torch.Generator().manual_seed(2023)
    )

    # create dataloaders with train+val support
    trainloaders = []
    valloaders = []
    for trainset_ in trainsets:
        num_total = len(trainset_)
        num_val = int(val_ratio * num_total)
        num_train = num_total - num_val

        for_train, for_val = random_split(
            trainset_, [num_train, num_val], torch.Generator().manual_seed(2023)
        )
        trainloaders.append(
            DataLoader(for_train, batch_size=batch_size, shuffle=True, num_workers=0)
        )
        valloaders.append(
            DataLoader(for_val, batch_size=batch_size, shuffle=False, num_workers=0)
        )

    # create dataloader for the test set
    testloader = DataLoader(testset, batch_size=128)

    datapoint_count = len(trainset[0][0])
    print('Minimum DataPoint required for a signal :', datapoint_count)
    return trainloaders, valloaders, testloader

# Define CNN model for ECG classification
class ecg_net(nn.Module):
    def __init__(self, num_of_class):
        super(ecg_net, self).__init__()
        self.model = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),
        )
        self.linear = nn.Sequential(
            nn.Linear(2944, 500),
            nn.LeakyReLU(inplace=True),
            nn.Linear(500, num_of_class),
        )

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.model(x)
        x = x.view(x.size(0), -1)
        return self.linear(x)

# Evaluate model performance on a loader
def evalute(model, loader):
    model.eval()
    correct = 0
    total = len(loader)
    val_bar = tqdm(loader, file=sys.stdout)
    for x, y in val_bar:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
        correct += torch.eq(pred, y).sum().float().item()
    return correct / total

# Train the model on client's local data
def train_client(model, train_loader, valid_loader, epochs=1):
    optimizer = optim.Adam(model.parameters(), lr=3e-3)
    criteon = nn.CrossEntropyLoss()
    best_acc, best_epoch, global_step = 0, 0, 0
    for epoch in range(epochs):
        train_bar = tqdm(train_loader, file=sys.stdout)
        for step, (x, y) in enumerate(train_bar):
            x, y = x.to(device), y.to(device)
            model.train()
            logits = model(x)
            loss = criteon(logits, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_bar.desc = f"train epoch[{epoch+1}/{epochs}] loss:{loss:.3f}"
            global_step += 1
        val_acc = evalute(model, valid_loader)
        if val_acc > best_acc:
            best_epoch = epoch
            best_acc = val_acc
    print('best acc:', best_acc, 'best epoch:', best_epoch)

# Define Flower client
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, trainloader, vallodaer) -> None:
        super().__init__()
        self.trainloader = trainloader
        self.valloader = vallodaer
        self.model = ecg_net(2).to(device)

    def set_parameters(self, parameters):
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
        self.model.load_state_dict(state_dict, strict=True)

    def get_parameters(self, config: Dict[str, Scalar]):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        train_client(self.model, self.trainloader, self.valloader, epochs=5)
        return self.get_parameters({}), len(self.trainloader), {}

    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar]) -> Tuple[float, int, Dict[str, Scalar]]:
        self.set_parameters(parameters)
        criterion = nn.CrossEntropyLoss()
        total_loss = 0.0
        correct = 0
        total = 0
        self.model.eval()
        with torch.no_grad():
            for inputs, labels in self.valloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        avg_loss = total_loss / len(self.valloader)
        accuracy = correct / total
        return avg_loss, total, {"accuracy": accuracy}

# Server strategy and training setup
model = ecg_net(2).to(device)

# Evaluation function for server
def evaluate(server_round, parameters, config):
    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)
    criterion = nn.CrossEntropyLoss()
    total_loss, total_correct, total_samples = 0.0, 0, 0
    model.eval()
    with torch.no_grad():
        for valloader in valloaders:
            for inputs, labels in valloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                total_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total_correct += (predicted == labels).sum().item()
                total_samples += labels.size(0)
    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples
    return avg_loss, {"accuracy": accuracy}

# Strategy with model saving
class SaveModelStrategy(fl.server.strategy.FedAvg):
    def aggregate_fit(self, server_round, results, failures):
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)
        if aggregated_parameters is not None:
            print(f"Saving round {server_round} aggregated weights...")
            aggregated_ndarrays = fl.common.parameters_to_ndarrays(aggregated_parameters)
            np.savez(f"round-{server_round}-weights.npz", *aggregated_ndarrays)
            params_dict = zip(model.state_dict().keys(), aggregated_ndarrays)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            model.load_state_dict(state_dict, strict=True)
            torch.save(model.state_dict(), f"model_round_{server_round}.pth")
        return aggregated_parameters, aggregated_metrics

# Define config and metrics aggregation
def fit_config(server_round):
    return {"batch_size": 1, "local_epochs": 1 if server_round < 2 else 2}

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]
    return {"accuracy": sum(accuracies) / sum(examples)}

# Client generator
def generate_client_fn(trainloaders, valloaders):
    def client_fn(cid: str):
        return FlowerClient(trainloader=trainloaders[int(cid)], vallodaer=valloaders[int(cid)])
    return client_fn

# Dataset preparation and training launch
NUM_CLIENTS = 2
trainloaders, valloaders, testloader = prepare_dataset(num_partitions=NUM_CLIENTS, batch_size=20, val_ratio=0.1)
client_fn_callback = generate_client_fn(trainloaders, valloaders)
strategy = SaveModelStrategy(
    fraction_fit=1.0,
    min_fit_clients=2,
    min_available_clients=2,
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,
)

start_training = time.time()
fl.simulation.start_simulation(
    client_fn=client_fn_callback,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1},
    ray_init_args={"log_to_driver": False, "num_cpus": 1, "num_gpus": 1}
)
end_training = time.time()

print('Total Training Time:', end_training - start_training)
current, peak = tracemalloc.get_traced_memory()
print(f'Current memory [MB]: {current/(1024*1024):.2f}, Peak memory [MB]: {peak/(1024*1024):.2f}')
tracemalloc.stop()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NameError: name 'get_dataset' is not defined

In [ ]:
# -*- coding: utf-8 -*-
"""Federated Learning Simulation for ECG Classification using CNN"""

# Mount Google Drive to access datasets
from google.colab import drive
drive.mount('/content/drive')

# Install required packages
!pip install h5py typing-extensions wheel
!pip install -U flwr["simulation"]

# Import system monitoring and timing utilities
import tracemalloc
import time

# Start memory tracking
tracemalloc.start()

# Import necessary libraries
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler, TensorDataset
from torch import nn, optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm import tqdm

# Set hyperparameters
batch_size = 500
lr = 3e-3
epochs = 21
val_split_factor = 0.2
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Dataset preparation function
def prepare__dataset(abnormal: pd.DataFrame, normal: pd.DataFrame, val_split_factor: float) -> tuple:
    """
    Prepares training and validation datasets by combining abnormal and normal ECG samples.

    Args:
        abnormal (pd.DataFrame): Abnormal ECG samples.
        normal (pd.DataFrame): Normal ECG samples.
        val_split_factor (float): Fraction of the dataset to reserve for validation.

    Returns:
        Tuple containing training set, validation set, and example counts.
    """
    # Drop label column (assumed to be at index 187)
    abnormal = abnormal.drop([187], axis=1)
    normal = normal.drop([187], axis=1)

    # Create target labels (1 for abnormal, 0 for normal)
    y_abnormal = pd.DataFrame(np.ones((abnormal.shape[0])))
    y_normal = pd.DataFrame(np.zeros((normal.shape[0])))

    # Combine data and labels
    x = pd.concat([abnormal, normal], sort=True).to_numpy()
    y = pd.concat([y_abnormal, y_normal], sort=True)[0].to_numpy()

    # Convert to tensor dataset
    full_dataset = TensorDataset(torch.from_numpy(x).float(), torch.from_numpy(y).long())

    # Shuffle and split dataset
    full_len = len(full_dataset)
    val_len = int(full_len * val_split_factor)
    train_len = full_len - val_len
    indices = torch.randperm(full_len).tolist()
    train_indices = indices[:train_len]
    val_indices = indices[train_len:]

    # Create samplers and data loaders
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)
    train_loader = DataLoader(full_dataset, batch_size=train_len, sampler=train_sampler)
    val_loader = DataLoader(full_dataset, batch_size=val_len, sampler=val_sampler)

    # Extract single batch (entire dataset at once)
    train_dataset = list(train_loader)[0]
    val_dataset = list(val_loader)[0]

    num_examples = {'trainset': train_len, 'testset': val_len}

    return train_dataset, val_dataset, num_examples

# Load ECG datasets from Google Drive
root = '/content/drive/MyDrive/Federated-ECG/ECG_Classification/client/datasets/'
abnormal = pd.read_csv(root + 'ptbdb_abnormal.csv', header=None)
normal = pd.read_csv(root + 'ptbdb_normal.csv', header=None)

# Prepare training and validation datasets
train_dataset, val_dataset, _ = prepare__dataset(abnormal=abnormal, normal=normal, val_split_factor=val_split_factor)

# Print dataset shapes for verification
print("Abnormal samples shape:", abnormal.shape)
print("Normal samples shape:", normal.shape)

# Define 1D CNN model for ECG classification
class ecg_net(nn.Module):
    def __init__(self, num_of_class: int):
        super(ecg_net, self).__init__()

        # Convolutional feature extractor
        self.model = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),

            nn.Conv1d(16, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),

            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),
        )

        # Fully connected layers for classification
        self.linear = nn.Sequential(
            nn.Linear(2944, 500),
            nn.LeakyReLU(inplace=True),
            nn.Linear(500, num_of_class),
        )

    def forward(self, x):
        x = x.unsqueeze(1)              # Add channel dimension: [B, 1, 187]
        x = self.model(x)               # Pass through CNN layers
        x = x.view(x.size(0), -1)       # Flatten features
        x = self.linear(x)              # Classification head
        return x


"""Federated Learning Simulation: One Client, One Data Partition

In FL, each client must work with its own local data. This script simulates that setup by:
1. Preparing and combining ECG data.
2. Splitting it into N partitions (one per client).
3. Creating train/val/test loaders per client for simulation purposes.
"""

import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def create_combined_dataset_from_raw(abnormal_df, normal_df, val_split_ratio):
    """
    Combines abnormal and normal ECG samples, assigns labels, and returns train/validation datasets.

    Args:
        abnormal_df (pd.DataFrame): Abnormal ECG data.
        normal_df (pd.DataFrame): Normal ECG data.
        val_split_ratio (float): Fraction of data reserved for validation.

    Returns:
        Tuple: train_dataset, val_dataset, and dictionary of data sizes.
    """
    # Remove the label column (assumed at index 187)
    abnormal_df = abnormal_df.drop([187], axis=1)
    normal_df = normal_df.drop([187], axis=1)

    # Label assignment: 1 = abnormal, 0 = normal
    y_abnormal = pd.DataFrame(np.ones((abnormal_df.shape[0])))
    y_normal = pd.DataFrame(np.zeros((normal_df.shape[0])))

    # Concatenate features and labels
    x = pd.concat([abnormal_df, normal_df], sort=True).to_numpy()
    y = pd.concat([y_abnormal, y_normal], sort=True)[0].to_numpy()

    # Wrap into a TensorDataset
    full_dataset = TensorDataset(torch.from_numpy(x).float(), torch.from_numpy(y).long())

    # Split into training and validation
    total_len = len(full_dataset)
    val_len = int(total_len * val_split_ratio)
    train_len = total_len - val_len

    train_dataset, val_dataset = random_split(full_dataset, [train_len, val_len])

    return train_dataset, val_dataset, {'trainset': train_len, 'valset': val_len}


def load_and_split_data_from_csv(train_path, test_path, batch_size=100, val_split_ratio=0.2):
    """
    Loads preprocessed ECG datasets from CSV files and returns data loaders.

    Args:
        train_path (str): Path to training CSV.
        test_path (str): Path to test CSV.
        batch_size (int): Batch size for DataLoaders.
        val_split_ratio (float): Fraction of training data to use for validation.

    Returns:
        Tuple: train_loader, val_loader, test_loader, and data counts.
    """
    train_df = pd.read_csv(train_path, header=None)
    test_df = pd.read_csv(test_path, header=None)

    # Convert to NumPy arrays
    train_data = train_df.to_numpy()
    test_data = test_df.to_numpy()

    # Create TensorDatasets
    train_dataset = TensorDataset(torch.from_numpy(train_data[:, :-1]).float(),
                                  torch.from_numpy(train_data[:, -1]).long())
    test_dataset = TensorDataset(torch.from_numpy(test_data[:, :-1]).float(),
                                 torch.from_numpy(test_data[:, -1]).long())

    # Train/validation split
    total_len = len(train_dataset)
    val_len = int(total_len * val_split_ratio)
    train_len = total_len - val_len
    train_subset, val_subset = random_split(train_dataset, [train_len, val_len])

    # DataLoaders
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    return train_loader, val_loader, test_loader, {'trainset': train_len, 'valset': val_len}


def simulate_fl_dataset_partitioning(num_clients, batch_size, val_ratio=0.1):
    """
    Simulates FL environment by partitioning the training set into local datasets for each client.

    Each client receives a disjoint train+val split, and the test set is shared for evaluation.

    Args:
        num_clients (int): Number of simulated clients.
        batch_size (int): Batch size per client DataLoader.
        val_ratio (float): Portion of each client’s local data used for validation.

    Returns:
        Tuple: List of trainloaders, valloaders for all clients, and a testloader.
    """

    # Use global abnormal/normal data assumed to be already loaded
    global abnormal, normal, val_split_factor

    # Step 1: Combine data and perform initial train/val split
    full_trainset, global_valset, _ = create_combined_dataset_from_raw(
        abnormal_df=abnormal,
        normal_df=normal,
        val_split_ratio=val_split_factor
    )

    print("Total training examples after initial split:", len(full_trainset))
    print("Global validation examples:", len(global_valset))

    # Step 2: Divide the training set among clients
    partition_size = len(full_trainset) // num_clients
    lengths = [partition_size] * num_clients
    lengths[-1] += len(full_trainset) - sum(lengths)  # Adjust for rounding

    client_trainsets = random_split(full_trainset, lengths, generator=torch.Generator().manual_seed(2023))
    print("Partition sizes per client:", [len(p) for p in client_trainsets])

    # Step 3: Per-client train/val split + DataLoaders
    trainloaders, valloaders = [], []

    for client_data in client_trainsets:
        total_len = len(client_data)
        val_len = int(total_len * val_ratio)
        train_len = total_len - val_len

        local_train, local_val = random_split(client_data, [train_len, val_len],
                                              generator=torch.Generator().manual_seed(2023))

        trainloaders.append(DataLoader(local_train, batch_size=batch_size, shuffle=True))
        valloaders.append(DataLoader(local_val, batch_size=batch_size, shuffle=False))

    # Step 4: Create a test loader from the global validation set
    testloader = DataLoader(global_valset, batch_size=128)

    # Log example datapoint size
    print('Sample input vector length:', len(full_trainset[0][0]))

    return trainloaders, valloaders, testloader


# ========== Example Usage ========== #

# Define global variables
val_split_factor = 0.2
NUM_CLIENTS = 2

# Load dataset (replace with actual CSV paths if needed)
root = '/content/drive/MyDrive/Federated-ECG/ECG_Classification/client/datasets/'
abnormal = pd.read_csv(root + 'ptbdb_abnormal.csv', header=None)
normal = pd.read_csv(root + 'ptbdb_normal.csv', header=None)

# Prepare the client loaders
trainloaders, valloaders, testloader = simulate_fl_dataset_partitioning(
    num_clients=NUM_CLIENTS,
    batch_size=20,
    val_ratio=0.1
)


# !pip install -U flwr["simulation"]

import flwr as fl

# !cp /content/drive/MyDrive/TNR\ Lab/Federated-ECG/simulate_fl/client_train.py /content

"""Model"""

import torch
from torch.utils.data import DataLoader
from torch import nn,optim
import sys
from tqdm import tqdm
import pandas as pd
import numpy as np

#define the ecg_net model
class ecg_net(nn.Module):

    def __init__(self, num_of_class):
        super(ecg_net, self).__init__()

        self.model = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),

            nn.Conv1d(16, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),

            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),

        )

        self.linear = nn.Sequential(
            nn.Linear(2944,500),
            nn.LeakyReLU(inplace=True),
            nn.Linear(500, num_of_class),

        )


    def forward(self,x):
        x = x.unsqueeze(1)
        x = self.model(x)
        # print(x.shape)
        x = x.view(x.size(0), -1)
        #x [b, 2944]
        # print(x.shape)
        x = self.linear(x)

        return x


# hyperparameters
batch_size=1
lr = 3e-3
epochs = 10
val_split_factor = 0.2
torch.manual_seed(1234)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))

"""Client"""

def evalute(model, loader):
    model.eval()

    correct = 0
    total = len(loader)
    val_bar = tqdm(loader, file=sys.stdout)
    for x, y in val_bar:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
        correct += torch.eq(pred, y).sum().float().item()

    return correct / total

def train_client(model, train_loader, valid_loader, epochs=1):

    # model = ecg_net(2).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criteon = nn.CrossEntropyLoss()

    best_acc, best_epoch = 0, 0
    global_step = 0


    for epoch in range(epochs):

        train_bar = tqdm(train_loader, file=sys.stdout)
        for step, (x, y) in enumerate(train_bar):
            # x: [b, 187], y: [b]
            x, y = x.to(device), y.to(device)

            model.train()

            logits = model(x)
            loss = criteon(logits, y)

            optimizer.zero_grad()
            loss.backward()

            # for param in model.parameters():
            #     print(param.grad)

            optimizer.step()

            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,
                                                                     epochs,
                                                                     loss)

            global_step += 1

        if epoch % 1 == 0:  # You can change the validation frequency as you wish

            val_acc = evalute(model, valid_loader)

            print('val_acc = ',val_acc)
            if val_acc > best_acc:
                best_epoch = epoch
                best_acc = val_acc

                # torch.save(model.state_dict(), 'best_client_model.mdl')

        print("Global steps", global_step)

    print('best acc:', best_acc, 'best epoch:', best_epoch)

from collections import OrderedDict
from typing import Dict, List, Tuple
import torch
from flwr.common import NDArrays, Scalar

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, trainloader, vallodaer) -> None:
        super().__init__()

        self.trainloader = trainloader
        self.valloader = vallodaer
        self.model = ecg_net(2)
        # Determine device
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)  # send model to device

    def set_parameters(self, parameters):
        """With the model paramters received from the server,
        overwrite the uninitialise model in this class with them."""

        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
        # now replace the parameters
        self.model.load_state_dict(state_dict, strict=True)

    def get_parameters(self, config: Dict[str, Scalar]):
        """Extract all model parameters and conver them to a list of
        NumPy arryas. The server doesn't work with PyTorch/TF/etc."""
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def fit(self, parameters, config):
        """This method train the model using the parameters sent by the
        server on the dataset of this client. At then end, the parameters
        of the locally trained model are communicated back to the server"""

        # copy parameters sent by the server into client's local model
        self.set_parameters(parameters)

        # read from config
        lr = 0.001 # config["lr"]
        epochs = 5 #config["epochs"]

        # Define the optimizer
        optim = torch.optim.SGD(self.model.parameters(), lr=lr, momentum=0.9)

        # do local training
        train_client(self.model, self.trainloader, self.valloader, epochs=epochs)

        # return the model parameters to the server as well as extra info (number of training examples in this case)
        return self.get_parameters({}), len(self.trainloader), {}

    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar]) -> Tuple[float, int, Dict[str, Scalar]]:
        """Evaluate the model on the locally held dataset."""

        # Update local model with parameters received from the server
        self.set_parameters(parameters)

        # Get the loss criterion
        criterion = torch.nn.CrossEntropyLoss()
        total_loss = 0.0
        correct = 0
        total = 0

        # Switch the model to evaluation mode
        self.model.eval()

        # Disable gradient calculation during evaluation
        with torch.no_grad():
            for inputs, labels in self.valloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Calculate average loss and accuracy
        avg_loss = total_loss / len(self.valloader)
        accuracy = correct / total

        # Return evaluation results
        return avg_loss, total, {"accuracy": accuracy}

model = ecg_net(2).to(device=device)
client = FlowerClient(trainloaders[0], valloaders[0])

"""Server"""

from typing import Dict, Optional, Tuple, List, Union
from collections import OrderedDict
import flwr as fl
from flwr.common import Scalar, Metrics
import torch
import torch.nn as nn
import numpy as np
import time
import tracemalloc


# ------------------ Evaluation Function ------------------

def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, Scalar],
) -> Optional[Tuple[float, Dict[str, Scalar]]]:
    """
    Custom evaluation function executed on the server after each round.
    Aggregates validation results from all clients.
    """
    # Load global parameters into model
    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)

    criterion = nn.CrossEntropyLoss()
    total_loss, total_correct, total_samples = 0.0, 0, 0
    device = next(model.parameters()).device

    model.eval()
    with torch.no_grad():
        for valloader in valloaders:
            for inputs, labels in valloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                total_loss += loss.item() * inputs.size(0)
                total_correct += (outputs.argmax(1) == labels).sum().item()
                total_samples += labels.size(0)

    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples

    return avg_loss, {"accuracy": accuracy}


# ------------------ Per-Round Config ------------------

def fit_config(server_round: int) -> Dict[str, Scalar]:
    """
    Defines training config (e.g. local epochs, batch size) for each client per round.
    """
    return {
        "batch_size": 1,
        "local_epochs": 1 if server_round < 2 else 2,
    }


# ------------------ Metrics Aggregation ------------------

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    """
    Aggregates evaluation metrics (e.g. accuracy) across clients using weighted average.
    """
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    total_examples = sum(num_examples for num_examples, _ in metrics)
    return {"accuracy": sum(accuracies) / total_examples}


# ------------------ Custom Strategy with Model Saving ------------------

class SaveModelStrategy(fl.server.strategy.FedAvg):
    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]],
        failures: List[Union[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes], BaseException]],
    ) -> Tuple[Optional[fl.common.Parameters], Dict[str, Scalar]]:

        # Call base FedAvg to get aggregated model parameters
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)

        if aggregated_parameters is not None:
            print(f"Saving model for round {server_round}...")

            # Convert parameters to NumPy arrays and save
            aggregated_ndarrays: List[np.ndarray] = fl.common.parameters_to_ndarrays(aggregated_parameters)
            np.savez(f"round-{server_round}-weights.npz", *aggregated_ndarrays)

            # Load parameters into model and save state dict
            state_dict = OrderedDict({
                k: torch.tensor(v) for k, v in zip(model.state_dict().keys(), aggregated_ndarrays)
            })
            model.load_state_dict(state_dict, strict=True)
            torch.save(model.state_dict(), f"model_round_{server_round}.pth")

        return aggregated_parameters, aggregated_metrics


# ------------------ Client Generator ------------------

def generate_client_fn(trainloaders, valloaders):
    """
    Returns a client creation function to be used by Flower's virtual client engine.
    """
    def client_fn(cid: str):
        return FlowerClient(
            trainloader=trainloaders[int(cid)],
            vallodaer=valloaders[int(cid)]
        )
    return client_fn


# ------------------ Launch Simulation ------------------

# Instantiate the strategy with custom configuration
strategy = SaveModelStrategy(
    fraction_fit=1.0,
    min_fit_clients=2,
    min_available_clients=2,
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,
    evaluate_metrics_aggregation_fn=weighted_average,
)

# Generate clients and simulate training
client_fn_callback = generate_client_fn(trainloaders, valloaders)
client_resources = {"num_cpus": 1, "num_gpus": 1}

# Start memory tracking and training
tracemalloc.start()
start_training = time.time()

fl.simulation.start_simulation(
    client_fn=client_fn_callback,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=strategy,
    client_resources=client_resources,
    ray_init_args={"log_to_driver": False, "num_cpus": 1, "num_gpus": 1},
)

end_training = time.time()

# ------------------ Training Summary ------------------

print(f"Total Training Time: {end_training - start_training:.2f} seconds")

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage: {current / (1024**2):.2f} MB")
print(f"Peak memory usage: {peak / (1024**2):.2f} MB")

tracemalloc.stop()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Abnormal samples shape: (10518, 188)
Normal samples shape: (4052, 188)
Total training examples after initial split: 11656
Global validation examples: 2914
Partition sizes per client: [5828, 5828]
Sample input vector length: 187
using cpu device.


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=3, no round_timeout
2025-04-12 19:18:38,229	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 1.0, 'object_store_memory': 3993989529.0, 'node:172.28.0.12': 1.0, 'memory': 7987979060.0, 'GPU': 1.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClient

Saving model for round 1...


INFO :      fit progress: (1, 0.49853538033069206, {'accuracy': 0.7946735395189003}, 96.29185228700044)
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)
INFO :      aggregate_fit: received 2 results and 0 failures


Saving model for round 2...


INFO :      fit progress: (2, 0.4080029101496824, {'accuracy': 0.8316151202749141}, 189.9826537680001)
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)
INFO :      aggregate_fit: received 2 results and 0 failures


Saving model for round 3...


INFO :      fit progress: (3, 0.19175772078469597, {'accuracy': 0.9329896907216495}, 285.3318970850005)
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 286.53s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.49724871168533963
INFO :      		round 2: 0.39643089690418193
INFO :      		round 3: 0.18616001289336828
INFO :      	History (loss, centralized):
INFO :      		round 0: 0.7211126998527763
INFO :      		round 1: 0.49853538033069206
INFO :      		round 2: 0.4080029101496824
INFO :      		round 3: 0.19175772078469597
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'accuracy': [(1, 0.7946735395189003),
INFO :      	              (2, 0.8316151202749141),
INFO :      	              (3, 0.9329896907216495)]}
INFO :      	History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.28178

Total Training Time: 309.05 seconds
Current memory usage: 23.21 MB
Peak memory usage: 89.56 MB
